<a href="https://colab.research.google.com/github/hanyeob/fine_dust/blob/master/make_scaled_csv_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 불러오기

In [189]:
import numpy as np
import pandas as pd

In [190]:
isc_2_1='isc_2_1.csv'
isc_3_1='isc_3_1.csv'
isc_3_3='isc_3_3.csv'
nsc_1_1='nsc_1_1.csv'
nsc_1_2='nsc_1_2.csv'
nsc_2_5='nsc_2_5.csv'
sdc_1_1='sdc_1_1.csv'
sdc_2_1='sdc_2_1.csv'
sic_2_6='sic_2_6.csv'
sic_4_2='sic_4_2.csv'

choosefile = sic_4_2
address = 'https://raw.githubusercontent.com/hanyeob/fine_dust/master/csv%20files/' + choosefile

In [191]:
dataset = pd.read_csv(address)

In [192]:
dataset.columns

Index(['tt', '0.3_outdoor', '0.5_outdoor', '1.0_outdoor', '3.0_outdoor',
       '5.0_outdoor', '10.0_outdoor', '0.3_classroom', '0.5_classroom',
       '1.0_classroom', '3.0_classroom', '5.0_classroom', '10.0_classroom',
       'temp_outdoor', 'RH_outdoor', 'AH_outside', 'CO2_outdoor',
       'temp_classroom', 'RH_classroom', 'AH_classroom', 'CO2_classroom',
       'ACH', 'temp_meteo_425', 'RH_meteo_425', 'WD_meteo_425', 'WS_meteo_425',
       'temp_meteo_889', 'RH_meteo_889', 'WD_meteo_889', 'WS_meteo_889',
       'PM10_air_outdoor', 'PM2.5_air_outdoor', 'temp_air_outdoor',
       'RH_air_outdoor', 'AH_air_outdoor', 'CO2_air_outdoor',
       'VOCS_air_outdoor', 'Noise_air_outdoor', 'PM10_air_classroom',
       'PM2.5_air_classroom', 'temp_air_classroom', 'RH_air_classroom',
       'AH_air_classroom', 'CO2_air_classroom', 'VOCS_air_classroom',
       'Noise_air_classroom', 'ΔT'],
      dtype='object')

In [193]:
dataset = dataset[['tt', '0.3_outdoor', '0.5_outdoor', '1.0_outdoor', '3.0_outdoor',
       '5.0_outdoor', '10.0_outdoor', '0.3_classroom', '0.5_classroom',
       '1.0_classroom', '3.0_classroom', '5.0_classroom', '10.0_classroom',
       'CO2_outdoor','CO2_classroom']]

In [194]:
dataset['tt'] = pd.to_datetime(dataset['tt'], format="%d-%m-%Y %H:%M:%S")

In [195]:
dataset.isnull().sum()

tt                0
0.3_outdoor       0
0.5_outdoor       0
1.0_outdoor       0
3.0_outdoor       0
5.0_outdoor       0
10.0_outdoor      0
0.3_classroom     0
0.5_classroom     0
1.0_classroom     0
3.0_classroom     0
5.0_classroom     0
10.0_classroom    0
CO2_outdoor       0
CO2_classroom     0
dtype: int64

In [196]:
for i in reversed(range(0,len(dataset))): 
    if(np.isnan(dataset.loc[i,'CO2_classroom'])):
      if(i+1 < len(dataset)):
        dataset.loc[i,'CO2_classroom'] = dataset.loc[i+1,'CO2_classroom']

for i in range(0,len(dataset)): 
    if(np.isnan(dataset.loc[i,'CO2_classroom'])):
      if(i > 0):
        dataset.loc[i,'CO2_classroom'] = dataset.loc[i-1,'CO2_classroom']

## 미세먼지, 초미세먼지 단위 맞춤 (Scaling)

In [197]:
dataset['0.3_outdoor_scaled'] = dataset['0.3_outdoor'] * 0.3 ** 3
dataset['0.5_outdoor_scaled'] = dataset['0.5_outdoor'] * 0.5 ** 3
dataset['1.0_outdoor_scaled'] = dataset['1.0_outdoor'] * 1.0 ** 3
dataset['3.0_outdoor_scaled'] = dataset['3.0_outdoor'] * 3.0 ** 3
dataset['5.0_outdoor_scaled'] = dataset['5.0_outdoor'] * 5.0 ** 3
dataset['10.0_outdoor_scaled'] = dataset['10.0_outdoor'] * 10.0 ** 3

dataset['0.3_classroom_scaled'] = dataset['0.3_classroom'] * 0.3 ** 3
dataset['0.5_classroom_scaled'] = dataset['0.5_classroom'] * 0.5 ** 3
dataset['1.0_classroom_scaled'] = dataset['1.0_classroom'] * 1.0 ** 3
dataset['3.0_classroom_scaled'] = dataset['3.0_classroom'] * 3.0 ** 3
dataset['5.0_classroom_scaled'] = dataset['5.0_classroom'] * 5.0 ** 3
dataset['10.0_classroom_scaled'] = dataset['10.0_classroom'] * 10.0 ** 3

## 미세먼지/초미세먼지(내부,외부) 데이터 통합

In [198]:
dataset['PM2.5_outdoor'] = np.sum(dataset.loc[:,'0.3_outdoor_scaled':'1.0_outdoor_scaled'], axis=1)
dataset['PM10_outdoor'] =  np.sum(dataset.loc[:,'3.0_outdoor_scaled':'10.0_outdoor_scaled'], axis=1)

dataset['PM2.5_classroom'] = np.sum( dataset.loc[:,'0.3_classroom_scaled':'1.0_classroom_scaled'], axis=1)
dataset['PM10_classroom'] = np.sum( dataset.loc[:,'3.0_classroom_scaled':'10.0_classroom_scaled'], axis=1)

In [199]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

In [200]:
# PM_fig = go.Figure()
# from plotly.subplots import make_subplots
# PM_fig = make_subplots(specs=[[{"secondary_y": True}]])

# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM2.5_outdoor'],line=dict(color='red'), mode='lines',name='PM2.5_outdoors'))
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM10_outdoor'],line=dict(color='blue'),mode='lines',name='PM10_outdoors'),secondary_y=True,)
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM2.5_classroom'],line=dict(color='orange'),mode='lines',name='PM2.5_classroom'))
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM10_classroom'],line=dict(color='skyblue'),mode='lines',name='PM10_classroom'),secondary_y=True,)

# PM_fig.update_yaxes(title_text="PM 2.5", secondary_y=False)
# PM_fig.update_yaxes(title_text="PM10", secondary_y=True)

## 교실 내 변화량 column생성

In [201]:
for i in range(1,len(dataset)) :
    dataset.loc[i,'PM2.5_classroom_change'] = dataset.loc[i,'PM2.5_classroom'] - dataset.loc[i-1,'PM2.5_classroom']
    dataset.loc[i,'PM10_classroom_change'] = dataset.loc[i,'PM10_classroom'] - dataset.loc[i-1,'PM10_classroom']
    dataset.loc[i,'CO2_classroom_change'] = dataset.loc[i,'CO2_classroom'] - dataset.loc[i-1,'CO2_classroom']

dataset.loc[0,'PM2.5_classroom_change'] =dataset.loc[1,'PM2.5_classroom_change']
dataset.loc[0,'PM10_classroom_change']  =dataset.loc[1,'PM10_classroom_change'] 
dataset.loc[0,'CO2_classroom_change']   =dataset.loc[1,'CO2_classroom_change']  

In [202]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

# 스케일링 (일반화)


In [203]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled = scaler.fit_transform(dataset[
                ['CO2_outdoor','PM2.5_outdoor','PM10_outdoor',
               'CO2_classroom','PM2.5_classroom','PM10_classroom',
               'CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change']
              ])
scaled = pd.DataFrame(data=scaled,columns=['CO2_outdoor','PM2.5_outdoor','PM10_outdoor',
               'CO2_classroom','PM2.5_classroom','PM10_classroom',
               'CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change'])

In [204]:
scaled = pd.concat([dataset.tt, scaled], axis=1)

In [205]:
scaled

,tt,CO2_outdoor,PM2.5_outdoor,PM10_outdoor,CO2_classroom,PM2.5_classroom,PM10_classroom,CO2_classroom_change,PM2.5_classroom_change,PM10_classroom_change
0,2019-12-09 17:21:58,2.172880,-0.243034,0.529866,-0.342581,1.259666,0.056165,-0.014231,-0.017581,-0.794181
1,2019-12-09 17:22:58,2.095391,-0.212904,0.441144,-0.344607,1.257464,-0.076304,-0.014231,-0.017581,-0.794181
2,2019-12-09 17:23:58,1.862921,-0.240694,0.554274,-0.346633,1.247452,-0.084361,-0.014231,-0.094764,-0.048080
3,2019-12-09 17:24:58,0.868470,-0.247208,0.772440,-0.350686,1.237001,-0.156891,-0.028180,-0.099106,-0.434727
4,2019-12-09 17:25:58,0.429361,-0.256958,0.792566,-0.352712,1.187057,0.102730,-0.014231,-0.489424,1.557173
...,...,...,...,...,...,...,...,...,...,...
4093,2019-12-12 13:34:59,-0.578006,-1.237343,-0.558029,-0.107559,-0.375189,0.026883,-0.042129,-0.134481,0.009822
4094,2019-12-12 13:35:59,-0.539261,-1.242357,-0.657426,-0.121741,-0.409462,0.011391,-0.097924,-0.334531,-0.092669
4095,2019-12-12 13:36:59,-0.565091,-1.241022,-0.543312,-0.139976,-0.447493,-0.084935,-0.125822,-0.371687,-0.577431
4096,2019-12-12 13:37:59,-0.745900,-1.256271,-0.771644,-0.154158,-0.464358,-0.074883,-0.097924,-0.162493,0.060514


In [206]:
scaled.to_csv(choosefile+'_for_predict.csv')